# Classificando o dataset MNIST usando MLPs e CNNs

### Membros

* Gabriel Pessoa
* Ícaro Guerra
* Lucas Barros
* Matheus Pessoa
* Rafael Mota

## Introdução
Esse relatório detalha o processo experimental do desenvolvimento de uma solução para o problema de classificação de dígitos escritos manualmente do dataset MNIST, usando Redes Neurais dos tipos: Multilayer Perceptron (MLP) e Convolutional Neural Network (CNN). O dataset MNIST consiste em 70 mil imagens 28x28 dos dígitos de 0 a 9, sendo 60 mil samples de treinamento e 10 mil samples de teste.

## Bibliotecas Utilizadas
Para a implementação utilzaremos as seguintes bibliotecas:


In [53]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam

* O Numpy e o Pandas são usados para a representação dos dados e para a implementação de funções auxiliares.

* Tensorflow e Keras são usados para a implementação e treinamento das Redes Neurais.

## Carregamento dos Dados
O código a seguir carrega os dados do dataset, adapta ao formato desejado e define os parâmetros globais sobre os dados.

In [54]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Parâmetros de Treinamento
Aqui definimos os parâmetros de treinamento que serão utilizados para todas as redes neurais. A escolha da quantidade de epochs e batch_size foi feita para que tenhamos maior perfomance e mais dinamicidade no experimento com os treinamentos.

In [55]:
# Training parameters
epochs = 30
batch_size = 128
verbose = 0

metrics = ["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]

## Funções de Avaliação
Aqui definimos as duas funções que vamos usar para avaliar as soluções. A primeira função `accuracy_per_class` retorna a métrica de *accuracy* para cada uma das classes de resposta, que no nosso caso, são os possíveis dígitos de 0 a 9. A segunda função `accuracy_precision_recall` retorna as métricas *accuracy*, *precision* e *recall* total.

Um breve descrição das métricas citadas seria:

* *accuracy*: Proporção de predição corretas com o total de casos.
* *precision*: Proporção de predições positivas corretas com o total de predições positivas da classe.
* *recall*: Proporção de predições positivas corretas com o total de casos da classe.

In [56]:
def accuracy_per_class(model):
    y_pred = model.predict(x_test)

    correct = [0] * num_classes
    total = [0] * num_classes

    for real, pred in zip(y_test.argmax(axis=1), y_pred.argmax(axis=1)):
        if real == pred:
            correct[real] += 1
        total[real] += 1

    accuracies = []
    for correct, total in zip(correct, total):
        accuracies.append(correct / total)

    return pd.Series(data=accuracies, index=range(0,num_classes), name="Accuracy per class")

def accuracy_precision_recall(model):
    _, acc, prec, rec = model.evaluate(x_test, y_test,verbose=0)

    return pd.Series(data=[acc, prec, rec],index=["Accuracy", "Precision", "Recall"])

## Experimentação

A seguir começaremos a fase de experimentação, testando diferentes modelos de Redes Neurais e de aprendizado para achar uma solução para o problema de classificação do dataset MNIST.


### MLP 1
Primeiramente, vamos usar uma rede neural MLP simples com as seguintes camadas:
* Uma camada de entrada com um neuron para cada um dos pixels da imagem;
* Uma camada oculta com 50 neurons;
* Uma camada de saída com ativação softmax, com um neuron para cada possível saída, no nosso caso, 10;

Essa estrutura será reutilizada em futuras redes, podendo variar os parâmetros de aprendizado e a estrutura das camadas ocultas.

In [57]:
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(50),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_15 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 50)                39250     
_________________________________________________________________
dense_32 (Dense)             (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


In [58]:

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)


In [59]:
accuracy_per_class(model)

0    0.978571
1    0.977093
2    0.894380
3    0.909901
4    0.935845
5    0.867713
6    0.943633
7    0.915370
8    0.883984
9    0.888999
Name: Accuracy per class, dtype: float64

In [60]:
accuracy_precision_recall(model)

Accuracy     0.920600
Precision    0.939435
Recall       0.904300
dtype: float64

Como podemos observar, uma rede neural simples já acerta grande parte dos casos do Dataset.

### MLP 2
A seguir verificamos o impacto de aumentar a quantidade de neurons na mesma camada, conservando os demais parâmetros.

In [61]:
model2 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400),
    layers.Dense(num_classes, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model2.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_16 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_34 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [62]:
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [63]:
accuracy_precision_recall(model2)

Accuracy     0.920800
Precision    0.940401
Recall       0.905700
dtype: float64

Podemos verificar que não houve um ganho de desempenho significativo com o aumento de neurons na camada oculta (aproximadamente 0.001 de *precision* e *accuracy*)

### MLP 3
Podemos tentar também adicionar mais uma camada oculta de neurons:

In [64]:
model2 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(300),
    layers.Dense(100),
    layers.Dense(num_classes, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model2.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_17 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_36 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_37 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [65]:
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [66]:
accuracy_precision_recall(model2)

Accuracy     0.922300
Precision    0.939184
Recall       0.909600
dtype: float64

Verificamos que houve um pequeno ganho de perfomance (aumento de 0.001 na *accuracy* comparado com o anterior), e que o processo de treinamento e predição ficaram bem mais lentos.

### MLP 4
Outro parâmetro que podemos variar é a função de ativação. O default para a biblioteca é a função linear, que não é muito recomendada para problemas não linearmente separáveis. Vamos verificar com o a função sigmoid.

In [67]:
model3 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="sigmoid"),
    layers.Dense(num_classes, activation="softmax")
])

model3.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model3.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_18 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_39 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [68]:
model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [69]:
accuracy_precision_recall(model3)

Accuracy     0.907400
Precision    0.939803
Recall       0.877400
dtype: float64

Usando a função de ativação sigmoid houve uma piora significativa no desempenho.

### MLP 5
Vamos testar agora com a função de ativação ReLU.

In [70]:
model4 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model4.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model4.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_19 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_41 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [71]:
model4.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [72]:
accuracy_precision_recall(model4)

Accuracy     0.947800
Precision    0.962239
Recall       0.935200
dtype: float64

Com a função de ativação ReLU houve uma melhora considerável de desempenho em relação às execuções usando outras funções de ativação.

### MLP 6
Agora, podemos tentar adicionar uma nova camada na rede, dessa vez usando a função de ativação ReLU:


In [73]:
model5 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model5.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model5.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_20 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_43 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_44 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_________________________________________________________________


In [74]:
model5.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [75]:
accuracy_precision_recall(model5)

Accuracy     0.960900
Precision    0.969051
Recall       0.955000
dtype: float64

O tempo para treinamento e predição aumentou mas houve um melhora considerável na *accuracy* da rede usando 2 camadas de neurons com a função de ativação ReLU.

### MLP 7

Em seguida, podemos testar variar a taxa de aprendizado, o valor padrão para esse parâmetro na biblioteca Keras é 0.01, vamos experimentar aumentar esse valor:

In [76]:
model6 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model6.compile(loss="categorical_crossentropy", optimizer=SGD(0.2), metrics=metrics)

model6.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_21 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_46 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_47 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_________________________________________________________________


In [77]:
model6.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [78]:
accuracy_precision_recall(model6)

Accuracy     0.982200
Precision    0.982785
Recall       0.981900
dtype: float64

Obtemos uma melhora em relação ao resultado anterior. Isso provavelmente acontece por estarmos executando com uma quantidade limitada de épocas, fazendo com que uma taxa de ativação maior faça a rede neural convergir mais rápidamente.

### MLP 8

Podemos também usar uma técnica de regularização chamada de Drop-out, que no processo de treinamento "ignora" uma porcentagem dos neurons a cada iteração. Essa técnica tem como objetivo distribuir o "trabalho" de reconhecimento das classes entre diversos neurons, o que pode ajudar a reduzir o overfitting. 

In [79]:
model7 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model7.compile(loss="categorical_crossentropy", optimizer=SGD(0.2), metrics=metrics)

model7.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_22 (Flatten)         (None, 784)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 400)               314000    
_________________________________________________________________
dropout_8 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_50 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_______________________________________________

In [80]:
model7.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [81]:
accuracy_precision_recall(model7)

Accuracy     0.985100
Precision    0.987164
Recall       0.984400
dtype: float64

Obtemos uma melhora no desempenho ao usar essa técnica (aproximadamente 0.003 de *accuracy*).

### MLP 9

Podemos também tentar mudar o algoritmo de treinamento. Um possível algoritmo é o Adam:

In [82]:
model8 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model8.compile(loss="categorical_crossentropy", optimizer=Adam(0.2), metrics=metrics)

model8.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_23 (Flatten)         (None, 784)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 400)               314000    
_________________________________________________________________
dropout_10 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_53 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_______________________________________________

In [83]:
model8.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [84]:
accuracy_precision_recall(model8)

Accuracy     0.101
Precision    0.000
Recall       0.000
dtype: float64

Analisando os resultados, podemos deduzir que o algoritmo Adam não funciona com learning rates altos para esse conjunto de dados.

### MLP 10

Tentamos em seguida executar o com o algoritmo de aprendizado Adam com a learning rate default.

In [85]:
model9 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model9.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model9.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_24 (Flatten)         (None, 784)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 400)               314000    
_________________________________________________________________
dropout_12 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_56 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_______________________________________________

In [86]:
model9.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [87]:
accuracy_precision_recall(model9)

Accuracy     0.984300
Precision    0.985276
Recall       0.983700
dtype: float64

Tivemos praticamente os mesmos valores com o algoritmo de treinamento Adam, portanto não teve uma melhora significativa. 

### CNN 1

Agora vamos tentar usar uma Rede Neural Convolucional, que tem uma vantagem em relação às MLPS, pois ela considera as estrutura da matriz de entrada. 

Primeiramente, vamos usar uma CNN simples com apenas uma camada convolucional e uma camada de pooling.

In [88]:
model10 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model10.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model10.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 5408)              0         
_________________________________________________________________
dense_57 (Dense)             (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [89]:
model10.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1, verbose=verbose)

In [90]:
accuracy_precision_recall(model10)

Accuracy     0.981600
Precision    0.983743
Recall       0.980300
dtype: float64

Tivemos uma boa *accuracy* de inicio, mas ainda ainda mais baixa do que nosso MLP inicial.

### CNN 2

Podemos também aumentar a janela de convolução:

In [91]:
model14 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model14.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model14.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_26 (Flatten)         (None, 10816)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 10)                108170    
Total params: 108,810
Trainable params: 108,810
Non-trainable params: 0
_________________________________________________________________


In [92]:
model14.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [93]:
accuracy_precision_recall(model14)

Accuracy     0.982100
Precision    0.982878
Recall       0.981600
dtype: float64

Teve uma pequena melhora na acurácia ao aumentar o tamanho da janela.

### CNN 3

Podemos também variar o parâmetro de stride, que determina o quantos valores serão "pulados" para gerar as janelas de convolução.

In [94]:
model13 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), strides=(2, 2), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model13.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model13.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                11530     
Total params: 11,850
Trainable params: 11,850
Non-trainable params: 0
_________________________________________________________________


In [95]:
model13.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [96]:
accuracy_precision_recall(model13)

Accuracy     0.980100
Precision    0.981249
Recall       0.978600
dtype: float64

Tivemos uma melhora não significativa.

### CNN 4

Podemos também tentar adicionar mais uma camada de convolução:

In [97]:
model11 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model11.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model11.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
_________________________________________________

In [98]:
model11.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [99]:
accuracy_precision_recall(model11)

Accuracy     0.990700
Precision    0.990897
Recall       0.990600
dtype: float64

Adicionando uma camada de convolução obtivemos uma melhora considerável.

### CNN 5

Podemos também tentar adicionar uma camada de drop-out para os neurons de Saída.

In [100]:
model12 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model12.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model12.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1600)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 10)              

In [101]:
model12.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

In [102]:
accuracy_per_class(model12)

0    0.994898
1    1.000000
2    0.988372
3    0.994059
4    0.994908
5    0.988789
6    0.989562
7    0.994163
8    0.993840
9    0.984143
Name: Accuracy per class, dtype: float64

In [103]:
accuracy_precision_recall(model12)

Accuracy     0.992400
Precision    0.992499
Recall       0.992400
dtype: float64

Adicionando a camada de drop-out conseguimos a nossa melhor acurácia até agora.